# Day 4 Tutorials: Natural Language Processing in Humans and Machines

## NLP in Humans: Insights from Neuroscience research

# 1) Encoding models

## Change runtime to use a GPU

This tutorial is much faster when a GPU is available to run the computations.
In Google Colab you can request access to a GPU by changing the runtime type.
To do so, click the following menu options in Google Colab:

(Menu) "Runtime" -> "Change runtime type" -> "Hardware accelerator" -> "GPU".



### Setup Google Colab, download the data and install all required dependencies

Uncomment and run the following cell to download the required packages.



In [ ]:
!git config --global user.email "you@example.com" && git config --global user.name "Your Name"
!wget -O- http://neuro.debian.net/lists/jammy.us-ca.libre | sudo tee /etc/apt/sources.list.d/neurodebian.sources.list
!apt-key adv --recv-keys --keyserver hkps://keyserver.ubuntu.com 0xA5D32F012649A5A9 > /dev/null
!apt-get -qq update > /dev/null
!apt-get install -qq inkscape git-annex-standalone > /dev/null
!pip install -q voxelwise_tutorials

# this is what each command does:
# - Set up an email and username to use git, git-annex, and datalad (required to download the data)
# - Add NeuroDebian to the package sources
# - Update the gpg keys to use NeuroDebian
# - Update the list of available packages
# - Install Inkscape to use more features from Pycortex, and install git-annex to download the data
# - Install the tutorial helper package, and all the required dependencies

Now run the following cell to download the data for the tutorials.



In [ ]:
from voxelwise_tutorials.io import download_datalad

DATAFILES = [
    "features/motion_energy.hdf",
    "features/wordnet.hdf",
    "mappers/S01_mappers.hdf",
    "responses/S01_responses.hdf",
]

source = "https://gin.g-node.org/gallantlab/shortclips"
destination = "/content/shortclips"

for datafile in DATAFILES:
    local_filename = download_datalad(
        datafile,
        destination=destination,
        source=source
    )

Now run the following cell to set up the environment variables for the
tutorials and pycortex.



In [ ]:
import os
os.environ['VOXELWISE_TUTORIALS_DATA'] = "/content"

import sklearn
sklearn.set_config(assume_finite=True)

Your Google Colab environment is now set up for the voxelwise tutorials.



In [ ]:
%reset -f


# Compute the explainable variance

Before fitting any voxelwise model to fMRI responses, it is good practice to
quantify the amount of signal in the test set that can be predicted by an
encoding model. This quantity is called the *explainable variance*.

The measured signal can be decomposed into a sum of two components: the
stimulus-dependent signal and noise. If we present the same stimulus multiple
times and we record brain activity for each repetition, the stimulus-dependent
signal will be the same across repetitions while the noise will vary across
repetitions. In voxelwise modeling, the features used to model brain activity
are the same for each repetition of the stimulus. Thus, encoding models will
predict only the repeatable stimulus-dependent signal.

The stimulus-dependent signal can be estimated by taking the mean of brain
responses over repeats of the same stimulus or experiment. The variance of the
estimated stimulus-dependent signal, which we call the explainable variance, is
proportional to the maximum prediction accuracy that can be obtained by a
voxelwise encoding model in the test set.

Mathematically, let $y_i, i = 1 \dots N$ be the measured signal in a
voxel for each of the $N$ repetitions of the same stimulus and
$\bar{y} = \frac{1}{N}\sum_{i=1}^Ny_i$ the average brain response
across repetitions. For each repeat, we define the residual timeseries between
brain response and average brain response as $r_i = y_i - \bar{y}$. The
explainable variance (EV) is estimated as

\begin{align}\text{EV} = \frac{1}{N}\sum_{i=1}^N\text{Var}(y_i) - \frac{N}{N-1}\sum_{i=1}^N\text{Var}(r_i)\end{align}


In the literature, the explainable variance is also known as the *signal
power*. For more information, see these references [1]_ [2]_ [3]_.


## Path of the data directory



In [ ]:
from voxelwise_tutorials.io import get_data_home

directory = get_data_home(dataset="shortclips")
print(directory)

In [ ]:
# modify to use another subject
subject = "S01"

## Compute the explainable variance



In [ ]:
import numpy as np
from voxelwise_tutorials.io import load_hdf5_array

First, we load the fMRI responses on the test set, which contains brain
responses to ten (10) repeats of the same stimulus.



In [ ]:
import os

file_name = os.path.join(directory, 'responses', f'{subject}_responses.hdf')
Y_test = load_hdf5_array(file_name, key="Y_test")
print("(n_repeats, n_samples_test, n_voxels) =", Y_test.shape)

Then, we compute the explainable variance for each voxel.



In [ ]:
from voxelwise_tutorials.utils import explainable_variance

ev = explainable_variance(Y_test)
print("(n_voxels,) =", ev.shape)

## Map to subject flatmap

To better understand the distribution of explainable variance, we map the
values to the subject brain. This can be done with [pycortex](https://gallantlab.github.io/pycortex/), which can create interactive 3D
viewers to be displayed in any modern browser. ``pycortex`` can also display
flattened maps of the cortical surface to visualize the entire cortical
surface at once.

Here, we do not share the anatomical information of the subjects for privacy
concerns. Instead, we provide two mappers:

- to map the voxels to a (subject-specific) flatmap
- to map the voxels to the Freesurfer average cortical surface ("fsaverage")

The first mapper is 2D matrix of shape (n_pixels, n_voxels) that maps each
voxel to a set of pixel in a flatmap. The matrix is efficiently stored in a
``scipy`` sparse CSR matrix. The function ``plot_flatmap_from_mapper``
provides an example of how to use the mapper and visualize the flatmap.



In [ ]:
from voxelwise_tutorials.viz import plot_flatmap_from_mapper
import matplotlib.pyplot as plt

mapper_file = os.path.join(directory, 'mappers', f'{subject}_mappers.hdf')
plot_flatmap_from_mapper(ev, mapper_file, vmin=0, vmax=0.7)
plt.show()

This figure is a flattened map of the cortical surface. A number of regions
of interest (ROIs) have been labeled to ease interpretation. If you have
never seen such a flatmap, we recommend taking a look at a [pycortex brain
viewer](https://www.gallantlab.org/brainviewer/Deniz2019), which displays
the brain in 3D. In this viewer, press "I" to inflate the brain, "F" to
flatten the surface, and "R" to reset the view (or use the ``surface/unfold``
cursor on the right menu). Press "H" for a list of all keyboard shortcuts.
This viewer should help you understand the correspondence between the flatten
and the folded cortical surface of the brain.



On this flatmap, we can see that the explainable variance is mainly located
in the visual cortex, in early visual regions like V1, V2, V3, or in
higher-level regions like EBA, FFA or IPS. This is expected since this
dataset contains responses to a visual stimulus.



## Map to "fsaverage"

The second mapper we provide maps the voxel data to a Freesurfer
average surface ("fsaverage"), that can be used in ``pycortex``.

First, let's download the "fsaverage" surface.



In [ ]:
import cortex

surface = "fsaverage"

if not hasattr(cortex.db, surface):
    cortex.utils.download_subject(subject_id=surface,
                                  pycortex_store=cortex.db.filestore)
    cortex.db.reload_subjects()  # force filestore reload
    assert hasattr(cortex.db, surface)

Then, we load the "fsaverage" mapper. The mapper is a matrix of shape
(n_vertices, n_voxels), which maps each voxel to some vertices in the
fsaverage surface. It is stored as a sparse CSR matrix. The mapper is applied
with a dot product ``@`` (equivalent to ``np.dot``).



In [ ]:
from voxelwise_tutorials.io import load_hdf5_sparse_array

voxel_to_fsaverage = load_hdf5_sparse_array(mapper_file,
                                            key='voxel_to_fsaverage')
ev_projected = voxel_to_fsaverage @ ev
print("(n_vertices,) =", ev_projected.shape)

We can then create a ``Vertex`` object in ``pycortex``, containing the
projected data. This object can be used either in a ``pycortex`` interactive
3D viewer, or in a ``matplotlib`` figure showing only the flatmap.



In [ ]:
vertex = cortex.Vertex(ev_projected, surface, vmin=0, vmax=0.7, cmap='viridis')

In [ ]:
from cortex.testing_utils import has_installed

fig = cortex.quickshow(vertex, colorbar_location='right',
                       with_rois=has_installed("inkscape"))
plt.show()

## References

.. [1] Sahani, M., & Linden, J. F. (2003). How linear are auditory cortical
   responses?. Advances in neural information processing systems, 125-132.

.. [2] Hsu, A., Borst, A., & Theunissen, F. E. (2004). Quantifying
   variability in neural responses and its application for the validation of
   model predictions. Network: Computation in Neural Systems, 15(2), 91-109.

.. [3] Schoppe, O., Harper, N. S., Willmore, B. D., King, A. J., & Schnupp,
       J. W. (2016). Measuring the performance of neural models. Frontiers in
       computational neuroscience, 10, 10.




In [ ]:
%reset -f


# Fit a ridge model with wordnet features

In this example, we model the fMRI responses with semantic "wordnet" features,
manually annotated on each frame of the movie stimulus. The model is a
regularized linear regression model, known as ridge regression. Since this
model is used to predict brain activity from the stimulus, it is called a
(voxelwise) encoding model.

This example reproduces part of the analysis described in Huth et al (2012)
[1]_. See this publication for more details about the experiment, the wordnet
features, along with more results and more discussions.

*Wordnet features:* The features used in this example are semantic labels
manually annotated on each frame of the movie stimulus. The semantic labels
include nouns (such as "woman", "car", or "building") and verbs (such as
"talking", "touching", or "walking"), for a total of 1705 distinct category
labels. To interpret our model, labels can be organized in a graph of semantic
relationship based on the [Wordnet](https://wordnet.princeton.edu/) dataset.

*Summary:* We first concatenate the features with multiple temporal delays to
account for the slow hemodynamic response. We then use linear regression to fit
a predictive model of brain activity. The linear regression is regularized to
improve robustness to correlated features and to improve generalization
performance. The optimal regularization hyperparameter is selected over a
grid-search with cross-validation. Finally, the model generalization
performance is evaluated on a held-out test set, comparing the model
predictions to the corresponding ground-truth fMRI responses.


In [ ]:
# modify to use another subject
subject = "S01"

## Load the data

We first load the fMRI responses. These responses have been preprocessed as
described in [1]_. The data is separated into a training set ``Y_train`` and a
testing set ``Y_test``. The training set is used for fitting models, and
selecting the best models and hyperparameters. The test set is later used
to estimate the generalization performance of the selected model. The
test set contains multiple repetitions of the same experiment to estimate
an upper bound of the model prediction accuracy (cf. previous example).



In [ ]:
import os
import numpy as np
from voxelwise_tutorials.io import load_hdf5_array

file_name = os.path.join(directory, "responses", f"{subject}_responses.hdf")
Y_train = load_hdf5_array(file_name, key="Y_train")
Y_test = load_hdf5_array(file_name, key="Y_test")

print("(n_samples_train, n_voxels) =", Y_train.shape)
print("(n_repeats, n_samples_test, n_voxels) =", Y_test.shape)

If we repeat an experiment multiple times, part of the fMRI responses might
change. However the modeling features do not change over the repeats, so the
voxelwise encoding model will predict the same signal for each repeat. To
have an upper bound of the model prediction accuracy, we keep only the
repeatable part of the signal by averaging the test repeats.



In [ ]:
Y_test = Y_test.mean(0)

print("(n_samples_test, n_voxels) =", Y_test.shape)

We fill potential NaN (not-a-number) values with zeros.



In [ ]:
Y_train = np.nan_to_num(Y_train)
Y_test = np.nan_to_num(Y_test)

Then, we load the semantic "wordnet" features, extracted from the stimulus at
each time point. The features corresponding to the training set are noted
``X_train``, and the features corresponding to the test set are noted
``X_test``.



In [ ]:
feature_space = "wordnet"

file_name = os.path.join(directory, "features", f"{feature_space}.hdf")
X_train = load_hdf5_array(file_name, key="X_train")
X_test = load_hdf5_array(file_name, key="X_test")

print("(n_samples_train, n_features) =", X_train.shape)
print("(n_samples_test, n_features) =", X_test.shape)

## Define the cross-validation scheme

To select the best hyperparameter through cross-validation, we must define a
cross-validation splitting scheme. Because fMRI time-series are
autocorrelated in time, we should preserve as much as possible the temporal
correlation. In other words, because consecutive time samples are correlated,
we should not put one time sample in the training set and the immediately
following time sample in the validation set. Thus, we define here a
leave-one-run-out cross-validation split that keeps each recording run
intact.



In [ ]:
from sklearn.model_selection import check_cv
from voxelwise_tutorials.utils import generate_leave_one_run_out

# indice of first sample of each run
run_onsets = load_hdf5_array(file_name, key="run_onsets")
print(run_onsets)

We define a cross-validation splitter, compatible with ``scikit-learn`` API.



In [ ]:
n_samples_train = X_train.shape[0]
cv = generate_leave_one_run_out(n_samples_train, run_onsets)
cv = check_cv(cv)  # copy the cross-validation splitter into a reusable list

## Define the model

Now, let's define the model pipeline.

We first center the features, since we will not use an intercept. The mean
value in fMRI recording is non-informative, so each run is detrended and
demeaned independently, and we do not need to predict an intercept value in
the linear model.

However, we prefer to avoid normalizing by the standard deviation of each
feature. If the features are extracted in a consistent way from the stimulus,
their relative scale is meaningful. Normalizing them independently from each
other would remove this information. Moreover, the wordnet features are
one-hot-encoded, which means that each feature is either present (1) or not
present (0) in each sample. Normalizing one-hot-encoded features is not
recommended, since it would scale disproportionately the infrequent features.



In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=True, with_std=False)

Then we concatenate the features with multiple delays to account for the
hemodynamic response. Due to neurovascular coupling, the recorded BOLD signal
is delayed in time with respect to the stimulus onset. With different delayed
versions of the features, the linear regression model will weigh each delayed
feature with a different weight to maximize the predictions. With a sample
every 2 seconds, we typically use 4 delays [1, 2, 3, 4] to cover the
hemodynamic response peak. In the next example, we further describe this
hemodynamic response estimation.



In [ ]:
from voxelwise_tutorials.delayer import Delayer
delayer = Delayer(delays=[1, 2, 3, 4])

Finally, we use a ridge regression model. Ridge regression is a linear
regression with L2 regularization. The L2 regularization improves robustness
to correlated features and improves generalization performance. The L2
regularization is controlled by a hyperparameter ``alpha`` that needs to be
tuned for each dataset. This regularization hyperparameter is usually
selected over a grid search with cross-validation, selecting the
hyperparameter that maximizes the predictive performances on the validation
set. See the previous example for more details about ridge regression and
hyperparameter selection.

For computational reasons, when the number of features is larger than the
number of samples, it is more efficient to solve ridge regression using the
(equivalent) dual formulation [2]_. This dual formulation is equivalent to
kernel ridge regression with a linear kernel. Here, we have 3600 training
samples, and 1705 * 4 = 6820 features (we multiply by 4 since we use 4 time
delays), therefore it is more efficient to use kernel ridge regression.

With one target, we could directly use the pipeline in ``scikit-learn``'s
``GridSearchCV``, to select the optimal regularization hyperparameter
(``alpha``) over cross-validation. However, ``GridSearchCV`` can only
optimize a single score across all voxels (targets). Thus, in the
multiple-target case, ``GridSearchCV`` can only optimize (for example) the
mean score over targets. Here, we want to find a different optimal
hyperparameter per target/voxel, so we use the package [himalaya](https://github.com/gallantlab/himalaya) which implements a
``scikit-learn`` compatible estimator ``KernelRidgeCV``, with hyperparameter
selection independently on each target.



In [ ]:
from himalaya.kernel_ridge import KernelRidgeCV

``himalaya`` implements different computational backends,
including two backends that use GPU for faster computations. The two
available GPU backends are "torch_cuda" and "cupy". (Each backend is only
available if you installed the corresponding package with CUDA enabled. Check
the ``pytorch``/``cupy`` documentation for install instructions.)

Here we use the "torch_cuda" backend, but if the import fails we continue
with the default "numpy" backend. The "numpy" backend is expected to be
slower since it only uses the CPU.



In [ ]:
from himalaya.backend import set_backend
backend = set_backend("torch_cuda", on_error="warn")
print(backend)

To speed up model fitting on GPU, we use single precision float numbers.
(This step probably does not change significantly the performances on non-GPU
backends.)



In [ ]:
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

Since the scale of the regularization hyperparameter ``alpha`` is unknown, we
use a large logarithmic range, and we will check after the fit that best
hyperparameters are not all on one range edge.



In [ ]:
alphas = np.logspace(1, 20, 20)

We also indicate some batch sizes to limit the GPU memory.



In [ ]:
kernel_ridge_cv = KernelRidgeCV(
    alphas=alphas, cv=cv,
    solver_params=dict(n_targets_batch=500, n_alphas_batch=5,
                       n_targets_batch_refit=100))

Finally, we use a ``scikit-learn`` ``Pipeline`` to link the different steps
together. A ``Pipeline`` can be used as a regular estimator, calling
``pipeline.fit``, ``pipeline.predict``, etc. Using a ``Pipeline`` can be
useful to clarify the different steps, avoid cross-validation mistakes, or
automatically cache intermediate results. See the ``scikit-learn``
[documentation](https://scikit-learn.org/stable/modules/compose.html) for
more information.



In [ ]:
from sklearn.pipeline import make_pipeline
pipeline = make_pipeline(
    scaler,
    delayer,
    kernel_ridge_cv,
)

We can display the ``scikit-learn`` pipeline with an HTML diagram.



In [ ]:
from sklearn import set_config
set_config(display='diagram')  # requires scikit-learn 0.23
pipeline

## Fit the model

We fit on the training set..



In [ ]:
_ = pipeline.fit(X_train, Y_train)

..and score on the test set. Here the scores are the $R^2$ scores, with
values in $]-\infty, 1]$. A value of $1$ means the predictions
are perfect.

Note that since ``himalaya`` is implementing multiple-targets
models, the ``score`` method differs from ``scikit-learn`` API and returns
one score per target/voxel.



In [ ]:
scores = pipeline.score(X_test, Y_test)
print("(n_voxels,) =", scores.shape)

If we fit the model on GPU, scores are returned on GPU using an array object
specific to the backend we used (such as a ``torch.Tensor``). Thus, we need to
move them into ``numpy`` arrays on CPU, to be able to use them for example in
a ``matplotlib`` figure.



In [ ]:
scores = backend.to_numpy(scores)

## Plot the model prediction accuracy

To visualize the model prediction accuracy, we can plot it for each voxel on
a flattened surface of the brain. To do so, we use a mapper that is specific
to the each subject's brain. (Check previous example to see how to use the
mapper to Freesurfer average surface.)



In [ ]:
import matplotlib.pyplot as plt
from voxelwise_tutorials.viz import plot_flatmap_from_mapper

mapper_file = os.path.join(directory, "mappers", f"{subject}_mappers.hdf")
ax = plot_flatmap_from_mapper(scores, mapper_file, vmin=0, vmax=0.4)
plt.show()

We can see that the "wordnet" features successfully predict part of the
measured brain activity, with $R^2$ scores as high as 0.4. Note that
these scores are generalization scores, since they are computed on a test set
that was not used during model fitting. Since we fitted a model independently
in each voxel, we can inspect the generalization performances at the best
available spatial resolution: individual voxels.

The best-predicted voxels are located in visual semantic areas like EBA, or
FFA. This is expected since the wordnet features encode semantic information
about the visual stimulus. For more discussions about these results, we refer
the reader to the original publication [1]_.



# 2) Semantic projections

We talked about some limitations of word embeddings and how semantic projections can help find new insights. Let's explore this with actual code!

First, we download the fastText embeddings:

Here's a function to load the fastText embeddings we just downloaded:

In [ ]:
import numpy as np
import io

def load_fasttext_aligned_vectors(fname, skip_first_line=True):
    '''Return dictionary of word embeddings from file saved in fasttext format.

    Parameters:
    -----------
    fname : str
        Name of file containing word embeddings.
    skip_first_line : bool
        If True, skip first line of file. Should do this if first line of file
        contains metadata.

    Returns:
    --------
    data : dict
        Dictionary of word embeddings.
    '''
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    if skip_first_line:
        n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.array([float(token) for token in tokens[1:]])
    return data

Let's load the fastText embeddings:

In [ ]:
# Load fastText vectors into model object
filepath = 'wiki.en.align.vec'
model = load_fasttext_aligned_vectors(filepath)

Here is a function to compute cosine similarity:

In [ ]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(a, b):
   return dot(a, b)/(norm(a)*norm(b))

Let's compare a few words:

In [ ]:
word_1 = 'dolphin'
word_2 = 'shark'
word_3 = 'whale'

print(f'Cosine similarity between pairs of words')
print(f'({word_1},{word_2}) = {cos_sim(model[word_1], model[word_2])}')
print(f'({word_1},{word_3}) = {cos_sim(model[word_1], model[word_3])}')
print(f'({word_2},{word_3}) = {cos_sim(model[word_2], model[word_3])}')

What do you think of these values?

Compare these words with other animals, nouns that aren't animals, words that aren't nouns...

In [ ]:
word_ = ...

Let's compare these words along a specific semantic dimension, namely "size".

To do so, we will define a vector in the embedding space that represents the size dimension.

We first choose words on both ends of the size dimension:

In [ ]:
# List of words
list_large = ['large', 'big', 'huge']
list_small = ['small', 'little', 'tiny']

Now, we compute the difference vector between all pairs (large-small) words and average them:

In [ ]:
# fastText vectors have size 300
size_vector = np.zeros(300)

for word_large in list_large:
  for word_small in list_small:
    size_vector += model[word_large] - model[word_small]
size_vector /= len(list_large)*len(list_small)

We can now project these words on the size dimension!

In [ ]:
print('Semantic projection on size dimension:')
print(f'{word_1} = {dot(model[word_1], size_vector)}')
print(f'{word_2} = {dot(model[word_2], size_vector)}')
print(f'{word_3} = {dot(model[word_3], size_vector)}')

Do these results make sense? Compare other words and see:

In [ ]:
word_ = ...

print('Semantic projection on size dimension:')
print(f'{word_} = {dot(model[word_], size_vector)}')

Now compare these words (or other words) on other semantic dimensions!

In [ ]:
word_ = ...

list_ = ...

We will now use semantic projections with actual brain data! 🧠

I trained an encoding model with fastText features on a subject reading english narratives.

We thus have model weights that reflect the semantic information of each voxel in the fastText space. Let's download those weights:

In [ ]:
!gdown https://drive.google.com/uc?id=1JQzlNB-6z8Z5eEpkaD0q7gIZ-6tThSaX

Let's have a look at these weights:

In [ ]:
import numpy as np

voxel_weights = np.load('COL_voxel_weights.npz')
print(voxel_weights.files)

In [ ]:
voxel_weights['fT_en']

In [ ]:
voxel_weights['fT_en'].shape

What do these dimensions represent?

We can now project the weight of each voxel onto our size dimension!

In [ ]:
proj_weights = np.dot(voxel_weights['fT_en'].T, size_vector)
print(proj_weights)

In [ ]:
!gdown https://drive.google.com/uc?id=11wLAHPLDwkYhDVYXyzJvMtOpJM9f6sfv